In [2]:
from typing import List
import requests
import json
import pandas as pd
import numpy as np
import time
import sys
from collections.abc import MutableMapping,Iterable


def get_API_response(url):
    response_API = requests.get(url)
    print('Status Code-', response_API.status_code)
    print('Loading Data--')
    start = time.time()
    raw_data = response_API.text
    # Parse Json
    data = json.loads(raw_data)
    print('Time taken to load data-',round((time.time()-start),3))
    return data
# API key created under contact@rijumukherjee.com account
api_key = ''

In [12]:
# Get Historical Ether Price
url_string = "https://api.etherscan.io/api?module=stats&action=ethdailyprice&startdate={0}&enddate={1}&sort={2}&apikey={3}" 
start_date = '2021-01-01'
end_date = '2021-01-01'
order = 'asc'
url = url_string.format(start_date, end_date, order, api_key)

In [14]:
data = get_API_response(url)

Status Code- 200
Loading Data--
Time taken to load data- 0.0


In [15]:
data

{'status': '0',
 'message': 'NOTOK',
 'result': 'Sorry, it looks like you are trying to access an API Pro endpoint. Contact us to upgrade to API Pro.'}

In [16]:
# Get total ether supply
url= f"https://api.etherscan.io/api?module=stats&action=ethsupply&apikey={api_key}"


In [18]:
data = get_API_response(url)
data

Status Code- 200
Loading Data--
Time taken to load data- 0.0


{'status': '1', 'message': 'OK', 'result': '121499119436500000000000000'}